In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, AdamW
from transformers import BertModel, BertConfig
from tqdm import tqdm

In [3]:
# Step 1: Define the Dataset Class
class TokenDataset(Dataset):
    def __init__(self, file_path):
        with open(file_path, 'r') as f:
            # Read and split the data into tokens
            self.data = f.read().split(' ')
        
        # Create input-output pairs
        self.inputs = []
        self.labels = []
        
        for i in range(len(self.data) - 1):
            self.inputs.append(self.data[i])
            # Define the label: 0 for space, 1 for newline, 2 for other
            if self.data[i + 1] == "<space>":
                self.labels.append(0)  # Space
            elif self.data[i + 1] == "<newline>":
                self.labels.append(1)  # Newline
            else:
                self.labels.append(2)  # Other

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], self.labels[idx]

In [32]:
# Load the dataset
file_path = 'data/preprocessed_cpp_data.txt'  # Update this with your actual file path
dataset = TokenDataset(file_path)

# Create DataLoader
batch_size = 32  # or your defined batch size
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Get and print dataset and DataLoader size
dataset_size = len(dataset)
num_batches = (dataset_size + batch_size - 1) // batch_size

print(f"Total dataset size: {dataset_size}")
print(f"Batch size: {batch_size}")
print(f"Number of batches: {num_batches}")

Total dataset size: 77431
Batch size: 32
Number of batches: 2420


In [28]:
for inputs, labels in dataloader:
    print(inputs)
    print(labels)
    break

('/', '/', '<space>', 'Copyright', '<space>', '(', 'c', ')', '<space>', '2011', '<space>', 'The', '<space>', 'LevelDB', '<space>', 'Authors', '.', '<space>', 'All', '<space>', 'rights', '<space>', 'reserved', '.', '<newline>', '/', '/', '<space>', 'Use', '<space>', 'of', '<space>')
tensor([2, 0, 2, 0, 2, 2, 2, 0, 2, 0, 2, 0, 2, 0, 2, 2, 0, 2, 0, 2, 0, 2, 2, 1,
        2, 2, 0, 2, 0, 2, 0, 2])


In [31]:
print(dataloader.size)

AttributeError: 'DataLoader' object has no attribute 'size'

In [10]:
# Step 3: Define a BERT Model from Scratch
class BertForSequenceClassification(nn.Module):
    def __init__(self, num_labels=3):
        super(BertForSequenceClassification, self).__init__()
        # Define the BERT configuration
        self.config = BertConfig(
            vocab_size=30522,  # This is the vocabulary size for BERT
            hidden_size=768,  # The size of the hidden layers
            num_hidden_layers=12,  # Number of transformer blocks
            num_attention_heads=12,  # Number of attention heads
            intermediate_size=3072,  # Size of the feed-forward layer
            hidden_act='gelu',  # Activation function
            num_labels=num_labels  # Number of labels for classification
        )
        # Define BERT model
        self.bert = BertModel(self.config)
        # Define classification head
        self.classifier = nn.Linear(self.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]  # Get the pooled output
        logits = self.classifier(pooled_output)
        return logits

In [11]:
# Step 4: Set Device and Initialize Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertForSequenceClassification(num_labels=3).to(device)

In [13]:
# Step 5: Define Tokenizer and Optimizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')  # Use a pre-trained tokenizer
optimizer = AdamW(model.parameters(), lr=5e-5)

In [33]:
# Step 6: Training Loop
epochs = 3
model.train()  # Set the model to training mode

for epoch in range(epochs):
    total_loss = 0
    for batch_idx, (inputs, labels) in enumerate(tqdm(dataloader)):
        # Tokenize the inputs
        inputs = tokenizer(inputs, padding=True, truncation=True, return_tensors='pt', max_length=128).to(device)

        # Convert labels to tensor correctly
        labels = torch.tensor(labels).long().clone().detach().to(device)

        # Forward pass
        outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])
        loss_fn = nn.CrossEntropyLoss()
        loss = loss_fn(outputs, labels)
        total_loss += loss.item()

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print the loss for the current mini-batch
        print(f"Epoch [{epoch + 1}/{epochs}], Batch [{batch_idx + 1}/{len(dataloader)}], Loss: {loss.item()}")

    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch + 1}/{epochs}, Average Loss: {avg_loss}")


  0%|                                                                                         | 0/2420 [00:00<?, ?it/s]C:\Users\JovanAStro\AppData\Local\Temp\ipykernel_3120\4221759339.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels).long().clone().detach().to(device)
  0%|                                                                               | 1/2420 [00:02<1:40:13,  2.49s/it]

Epoch [1/3], Batch [1/2420], Loss: 0.9389889240264893


  0%|                                                                               | 2/2420 [00:05<1:50:26,  2.74s/it]

Epoch [1/3], Batch [2/2420], Loss: 0.8532786965370178


  0%|                                                                               | 3/2420 [00:08<2:02:19,  3.04s/it]

...

Epoch [1/3], Batch [30/2420], Loss: 0.7273212671279907


  1%|▉                                                                             | 31/2420 [01:39<2:30:16,  3.77s/it]

Epoch [1/3], Batch [31/2420], Loss: 0.8027989864349365


  1%|▉                                                                             | 31/2420 [01:41<2:10:02,  3.27s/it]


KeyboardInterrupt: 